In [ ]:
from glob import glob

tracklet_dirs = '/mnt/sun/levlevi/nba-plus-statvu-dataset/player-tracklets/'
tracklet_file_paths = glob(tracklet_dirs + '*.txt')

In [23]:
import pandas as pd

MIN_ARR_SIZE = 30
NAMES = ['frame', 'entity_id', 'x1', 'y1', 'width', 'height', 'conf']
example_fp = tracklet_file_paths[0]

def format_tracklets_for_reid(fp: str):
    # Read the CSV file with only the necessary columns
    df = pd.read_csv(fp, sep=',', names=NAMES, usecols=NAMES)

    # Initialize list to store rows for filtered dataframe
    rows = []

    # Iterate through each unique entity_id
    for entity_id, df_entity in df.groupby('entity_id'):
        # Initialize variables for tracking consecutive frames
        last_frame_idx = -1
        temp_entity_rows = []
        
        # Iterate through rows of the filtered dataframe
        for row in df_entity.itertuples(index=False):
            temp_frame_idx = row.frame
            
            # Check if current frame is consecutive to the last frame
            if last_frame_idx != -1 and temp_frame_idx != last_frame_idx + 1:
                # If not consecutive and length of collected rows is sufficient, store the rows
                if len(temp_entity_rows) >= MIN_ARR_SIZE:
                    temp_df = pd.DataFrame(temp_entity_rows, columns=['frame', 'x1', 'y1', 'width', 'height', 'conf'])
                    rows.append([entity_id, temp_df])
                # Reset the temporary list for a new tracklet
                temp_entity_rows = []
            
            # Add the current row to the temporary list
            temp_entity_rows.append([row.frame, row.x1, row.y1, row.width, row.height, row.conf])
            # Update last frame index
            last_frame_idx = temp_frame_idx
        
        # Check the last collected rows for the entity
        if len(temp_entity_rows) >= MIN_ARR_SIZE:
            temp_df = pd.DataFrame(temp_entity_rows, columns=['frame', 'x1', 'y1', 'width', 'height', 'conf'])
            rows.append([entity_id, temp_df])
            
    tracklets_df = pd.DataFrame(rows, columns=['entity_id', 'tracklet_dataframe'])
    return tracklets_df

In [25]:
# dataframe containing all tracklets
example_tracklets_df = format_tracklets_for_reid(example_fp)
example_tracklets_df.head()

,entity_id,tracklet_dataframe
0,0,frame x1 y1 width height conf 0...
1,1,frame x1 y1 width height conf ...
2,2,frame x1 y1 width height conf ...
3,3,frame x1 y1 width height conf ...
4,4,frame x1 y1 width height conf 0...
